# TODO

## BERT on e-mails

 - run BERT on each e-mail 

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForMaskedLM, DistilBertConfig
from transformers import BertTokenizer, BertModel, BertConfig

import torch
import numpy as np

import pickle
from tqdm import tqdm

from collections import Counter
import matplotlib.pyplot as plt

import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

In [ ]:
with open("emails_token_ids.pkl", "rb") as handle:
    ids = pickle.load(handle)

In [ ]:
torch.tensor(tokenizer.encode(email_body, add_special_tokens=True))

# Roadmap

## encode e-mails with BERT
  
  - for each e-mail, $e=(w_1, ..., w_n)$, encode it with BERT:<br>
    $BERT(e)= (BERT(w_1), ..., BERT(w_n))$
    
  - save in convenient format <br>
    $\rightarrow$ be careful that size doesn't explode <br>
    $\rightarrow$ how to load for training?
    
  - make sure that order correctly recoverable <br>
    $\rightarrow$ index e-mails for cheap and convenient flows during training?
    
## build classifier architecture

  - LSTM: reads encoded e-mails (token by token) and outputs a single representation <br>
    $\rightarrow$ LSTM's output for last token (average of both outputs for bidirectional)
    
  - classifier: simple feedforward layer which pools LSTM's representation into single value to apply sigmoid over<br>
    $\rightarrow$ goal is to force the LSTM to produce useful representations, i.e. give the classifier as little power as possible <br>
    
  - train on e-mail tuples: either from same author ($y=1$) or not ($y=0$) <br>
    $\rightarrow$ take $e_1$ and $e_2$ from **same** author label as samples with $y=1$ <br>
    $\rightarrow$ take $e_1$ and $e_2$ from **different** author label as samples with $y=0$ 

    
    
## create data

  - use snorkel's augmentation framework:
    - sample positive and negative data
    - use a mean field policy to create more negative than positive data 
    - each sample is `((e, e'), y)`
    
  - put data into pandas for convenience?  
  
  - translate:
    - e-mails into indices (based on original order)
    - corpus into a dict with `auth: [e_1, ..e_n]`
    
  - split into train, eval and test set <br>
    $\rightarrow$ based on characteristics of authors or e-mails?
  
  

# Flow
    
`emails.pkl` ✔

$~~~~~~~~~~\downarrow~~~~$ `BertTokenizer` ✔

`emails_token_ids.pkl` ✔

$~~~~~~~~~~\downarrow~~~~$ `BertModel` 

`emails_encoded.pkl`

$~~~\downarrow~~$ $~~~\downarrow~~~~~~$ &emsp;&emsp;&emsp;&emsp; $\downarrow~~$`emails_to_data` ✔

`train` `eval` &emsp;&emsp;&emsp;&emsp; `test`

$~~~\downarrow~~~~$ `train_cls` $~~~~~~~\downarrow$

`trained_model` $~~~~~~~~~~~~\downarrow$

$~~~\downarrow~~~~$ $~~~~~~~~~~~~~~~~~~~~~~~~~~~~\downarrow$ `test_classifier`

`accuracy` and `other_scores`